In [ ]:
pip install mip pandas openpyxl

In [1]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()

import pandas as pd

import os
import openpyxl

from mip import Model, xsum, BINARY, OptimizationStatus



# Input

In [ ]:
filepath_allocations = filedialog.askopenfilename(title="Select file listing the allocation optimisation results")
print(filepath_allocations)

# Read the 'Student_allocations' sheet from the 'allocation results' file into a pandas DataFrame
sheet_name = 'Student_allocations'
try:
    df_student_allocations = pd.read_excel(filepath_allocations, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

    
# Read the 'Supervisor_allocations' sheet from the 'allocation results' file into a pandas DataFrame
sheet_name = 'Supervisor_allocations'
try:
    df_supervisor_allocations = pd.read_excel(filepath_allocations, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

In [ ]:
filepath_projects = filedialog.askopenfilename(title="Select file listing topics (projects)")
print(filepath_projects)

sheet_name = 'Topics'
try:
    df_projects = pd.read_excel(filepath_projects, sheet_name=sheet_name)
    print("'Projects' Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")

#print(df_projects)


sheet_name = 'Subject Areas'
try:
    df_subject_areas = pd.read_excel(filepath_projects, sheet_name=sheet_name)
    print("'Subject Areas' Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")

#print(df_subject_areas)

# Prepare Data for 2nd Reader Allocation Optimisation

In [ ]:
'''
# --- Example INPUT data--- 
# Projects indexed 0..P-1
projects_ids = list(range(10))

# Lead supervisor for each project
project_leads = [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]

# Area of each project
project_areas = [0, 1, 2, 0, 1, 2, 1, 0, 2, 0]

# Supervisors who CANNOT second read and are EXCLUDED from balancing
no_second_reading = {1, 4}
'''

projects_allocated = []
for index, row in df_student_allocations.iterrows():
    project_code = row['Project']
    project_supervisor = row['Supervisor']
    project_student = row['Student']
    projects_allocated.append({"Code": project_code, "Supervisor": project_supervisor, "Student": project_student})


# Index of each allocated project
num_projects = len(projects_allocated)
projects_ids = list(range(num_projects))

print(num_projects)
print(projects_ids)


# Lead supervisor of each allocated project
project_leads = []
for project in projects_allocated:
    project_supervisor = project['Supervisor']
    supervisor_id = next(
        (i for i, entry in enumerate(supervisors) if entry["Supervisor"] == project_supervisor),
        None  # returned if not found
    )
    project_leads.append(supervisor_id)

print(project_leads)


# Subject area of each allocated project
subject_areas = []
for index, row in df_subject_areas.iterrows():
    subject_area = row['Subject Areas']
    subject_areas.append(subject_area)

print(subject_areas)


projects_all = []
for index, row in df_projects.iterrows():
    project_code = row['Code']
    project_supervisor = row['Supervisor']
    project_subject_area = row['Subject Area']
    projects_all.append({"Code": project_code, "Supervisor": project_supervisor, "Subject Area": project_subject_area})
 

# Area of each allocated project
project_areas_str = []
for project in projects_allocated:
    project_code = project['Code']
    subject_areas_str = next(
        (entry["Subject Area"] for entry in projects_all) if entry["Code"] == project_code),
        None  # returned if not found
    )
    subject_areas = subject_areas_str.split(";#")

    # Get first Topic Area in list
    subject_area = subject_areas[0]

    project_areas.append(subject_area)



In [ ]:
'''
# --- Example INPUT data ---
# Supervisors indexed 0..S-1
supervisors_ids = list(range(num_supervisors))

# Existing supervision load (already assigned)
supervisors_loads = [2, 3, 1, 0, 4]

# Each supervisor's subject areas of interest (set of area IDs)
supervisors_subject_areas_ids = [
    {0, 1},  # supervisor 0
    {1, 2},  # supervisor 1
    {0, 2},  # supervisor 2
    {0, 1},  # supervisor 3
    {1, 2}   # supervisor 4
]

# Supervisors who CANNOT second read and are EXCLUDED from balancing
no_second_reading = {1, 4}

'''

supervisors = []

for index, row in df_supervisor_allocations.iterrows():
    supervisor_name = row['Supervisor']
    supervisor_short = row['Supervisor_short']
    supervisor_load = row['N_students_allocated']
    supervisor_subject_areas_str = row['Subject_area']
    supervisor_subject_areas = supervisor_subject_areas_str.split(";#")
    supervisor_2nd_reader = row['2nd_reader']
    if supervisor_name != "TOPIC AREA":
        supervisors.append({'Supervisor': supervisor_name, 
                            'Supervisor_short': supervisor_short,
                            'Load': supervisor_load. 
                            'Subject Areas': supervisor_subject_areas,
                            '2nd Reader': supervisor_2nd_reader})

# Supervisors indexed 0..S-1
num_supervisors = len(supervisors)
supervisors_ids = list(range(num_supervisors))

print(num_supervisors)
print(supervisors_ids)


# Existing supervision load (already assigned)
supervisors_loads = []
for supervisor in supervisors:
    supervisors_loads.append(supervisor["Load"])

print(supervisors_loads)


# -----------------------------
# Example INPUT data
# Each supervisor's subject areas of interest (set of area IDs)
supervisors_subject_areas_ids = []
for supervisor in supervisors:
    supervisor_subject_areas = supervisor["Subject Areas"]
    indices = {subject_areas.index(subject) for subject in supervisor_subject_areas}
    supervisors_subject_areas_ids.append(indices)


print(supervisors_subject_areas_ids)


# Supervisors who CANNOT second read and are EXCLUDED from balancing
no_second_reading = []
for i, supervisor in enumerate(supervisors):
    if supervisor[]'2nd Reader'] == 0:
        no_second_reading.append(i)
no_second_reading = set(no_second_reading)

print(no_second_reading)


# 2nd Reader Allocation Optimisation

In [ ]:
# Weight for subject area interest penalty in stage 2
penalty_weight = 1

# -----------------------------------------------------
# Stage 1: minimise max load among eligible supervisors
# -----------------------------------------------------

m1 = Model()

S = len(supervisors_ids)
P = len(projects_ids)

# Decision vars: x[s][p] = 1 if s is second reader for p
x = [[m1.add_var(var_type=BINARY) for p in projects_ids] for s in supervisors_ids]

# Max load variable (only applies to eligible supervisors)
L = m1.add_var()

# Each project gets exactly one second reader
for p in projects_ids:
    m1 += xsum(x[s][p] for s in supervisors_ids) == 1

# No self-reading + no-second-reading supervisors
for p in projects_ids:
    m1 += x[project_leads[p]][p] == 0
    for s in no_second_reading:
        m1 += x[s][p] == 0

# Load constraints — only for eligible supervisors
for s in supervisors_ids:
    if s not in no_second_reading:
        total_load = supervisors_loads[s] + xsum(x[s][p] for p in projects_ids)
        m1 += total_load <= L

# Stage 1 objective: minimise L
m1.objective = L

m1.verbose = 0
status = m1.optimize()

if status not in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE):
    raise RuntimeError("Stage 1 failed to find a solution")

L_star = L.x
print(f"Stage 1: Minimum possible max load (eligible supervisors only) = {L_star}")



# --------------------------------------------
# Stage 2: fix L and minimise subject area of interest penalty
# --------------------------------------------

m2 = Model()

x2 = [[m2.add_var(var_type=BINARY) for p in projects_ids] for s in supervisors_ids]

# Each project exactly one second reader
for p in projects_ids:
    m2 += xsum(x2[s][p] for s in supervisors_ids) == 1

# No self-reading + no-second-reading supervisors
for p in projects_ids:
    m2 += x2[project_leads[p]][p] == 0
    for s in no_second_reading:
        m2 += x2[s][p] == 0

# Load constraints with fixed L_star (only for eligible supervisors)
for s in supervisors_ids:
    if s not in no_second_reading:
        total_load = supervisors_loads[s] + xsum(x2[s][p] for p in projects_ids)
        m2 += total_load <= L_star

# Penalty for out-of-interest assignments
penalty = xsum(
    penalty_weight * x2[s][p]
    for s in supervisors_ids
    for p in projects_ids
    if project_areas[p] not in supervisors_subject_areas_ids[s]
)

# Stage 2 objective: minimise penalty
m2.objective = penalty

m2.verbose = 0
status = m2.optimize()

if status in (OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE):
    print(f"\nStage 2: Interest penalty = {penalty.x}")
    print(f"Max load (eligible supervisors only) = {L_star}")
    for p in projects_ids:
        for s in supervisors_ids:
            if x2[s][p].x >= 0.99:
                in_interest = project_areas[p] in supervisors_subject_areas_ids[s]
                tag = "(in interest)" if in_interest else "(out of interest)"
                print(f"Project {p} -> Second reader {s} {tag}")
else:
    print("Stage 2 failed to find a solution")

In [ ]:
m = Model()

# Decision vars: x[s][p] = 1 if s is second reader of p
x = [[m.add_var(var_type=BINARY) for p in range(num_projects)] for s in range(num_supervisors)]

# Max load variable
L = m.add_var()

# Each project gets exactly one second reader
for p in range(num_projects):
    m += xsum(x[s][p] for s in range(num_supervisors)) == 1

# No self-reading
for p in range(num_projects):
    m += x[project_leads[p]][p] == 0

# Load constraints: supervision + second reading <= L
for s in range(num_supervisors):
    total_load = supervision_load[s] + xsum(x[s][p] for p in range(num_projects))
    m += total_load <= L

# Objective: minimize L + penalty for out-of-interest assignments
penalty = 0
for s in range(num_supervisors):
    for p in range(num_projects):
        if project_areas[p] not in supervisors_subject_areas_ids[s]:
            penalty += 0.1 * x[s][p]  # weight topic-fit less than load balance

m.objective = L + penalty

m.optimize()

print("Optimal max load:", L.x)
for p in range(num_projects):
    for s in range(num_supervisors):
        if x[s][p].x >= 0.99:
            print(f"Project {p} -> Second reader {s}")
